<a href="https://colab.research.google.com/github/koll-ai/control-meme-api/blob/main/Controlmeme_Colab_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install deps & run webui in separate process ( ~2 min)

In [1]:
!curl -Lo memfix.zip https://github.com/nolanaatama/sd-webui/raw/main/memfix.zip
!unzip /content/memfix.zip
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *
!pip install --upgrade fastapi==0.90.1
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
!git clone https://github.com/nolanaatama/sd-webui-tunnels /content/stable-diffusion-webui/extensions/sd-webui-tunnels
!git clone https://github.com/Mikubill/sd-webui-controlnet /content/stable-diffusion-webui/extensions/sd-webui-controlnet
!git clone https://github.com/fkunn1326/openpose-editor /content/stable-diffusion-webui/extensions/openpose-editor
!git clone https://github.com/yfszzx/stable-diffusion-webui-images-browser /content/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
!git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete /content/stable-diffusion-webui/extensions/a1111-sd-webui-tagcomplete
# Model Code
!curl -Lo /content/stable-diffusion-webui/models/Stable-diffusion/dreamshaper3.32bvcf.ckpt https://huggingface.co/Lykon/DreamShaper/resolve/main/Dreamshaper_3.32_baked_vae_clip_fix.ckpt
# ControlNet
!curl -Lo /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/control_canny.safetensors https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_canny-fp16.safetensors
!curl -Lo /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/control_depth.safetensors https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_depth-fp16.safetensors
!curl -Lo /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/control_hed-fp16.safetensors https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_hed-fp16.safetensors
!curl -Lo /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/control_mlsd-fp16.safetensors https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_mlsd-fp16.safetensors
!curl -Lo /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/control_normal-fp16.safetensors https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_normal-fp16.safetensors
!curl -Lo /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/control_openpose-fp16.safetensors https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_openpose-fp16.safetensors
!curl -Lo /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/control_scribble-fp16.safetensors https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_scribble-fp16.safetensors
!curl -Lo /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/control_seg-fp16.safetensors https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_seg-fp16.safetensors
import shutil
shutil.rmtree('/content/stable-diffusion-webui/embeddings')
%cd /content/stable-diffusion-webui
!git checkout 0cc0ee1
!git clone https://huggingface.co/nolanaatama/embeddings
# Web UI tunnel (use cloudflare by default)
# If cloudflare failed to start, we may try these options:
# Change '--cloudflared' to '--remotemoe' on the COMMANDLINE_ARGS line above to use remotemoe tunnel
# Change '--cloudflared' to '--localhostrun' to use localhost.run tunnel
# Change '--cloudflared' to '--share' to use gradio tunnel
!curl -Lo /content/stable-diffusion-webui/models/Lora/wanostyle_2.safetensors https://huggingface.co/nolanaatama/opwslora/resolve/main/onePieceWanoStyle_20.safetensors
!curl -Lo /content/stable-diffusion-webui/models/Lora/last.pt https://huggingface.co/closertodeath/dpepmkmp/resolve/main/last.pt
!curl -Lo /content/stable-diffusion-webui/models/Lora/anime_screencap_v2-000030.safetensors https://huggingface.co/nolanaatama/asslora/resolve/main/asslora.safetensors

!npm i -g localtunnel
!pip install flask
!pip install flask_cors



  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  761k  100  761k    0     0  10.7M      0 --:--:-- --:--:-- --:--:-- 10.7M
Archive:  /content/memfix.zip
  inflating: google-perftools_2.5-2.2ubuntu3_all.deb  
  inflating: libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb  
  inflating: libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb  
  inflating: libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb  
The following NEW packages will be installed:
  libunwind-dev
0 upgraded, 1 newly installed, 0 to remove and 22 not upgraded.
Need to get 472 kB of archives.
After this operation, 4,163 kB of additional disk space will be used.
Selecting previously unselected package libunwind-dev:amd64.
(Reading database ... 128215 files and directories currently installed.)
Preparing to unpack .../libunwind-dev_1.2.1-9build1

In [2]:
import PIL
PIL.__version__

'8.4.0'

# Now restart the kernel (execution > restart kernel) and execute the following


In [3]:

import torch
import cv2
import numpy as np
from PIL import Image

from flask import Flask
from flask import request, send_file
import base64
from io import BytesIO
import os

import subprocess
import time
from flask_cors import CORS

import requests


from io import BytesIO
import sys
sys.path.append('/content/stable-diffusion-webui/extensions/sd-webui-controlnet/scripts')
sys.path.append('/content/stable-diffusion-webui/extensions/sd-webui-controlnet')
import cv2
from processor import canny, simple_scribble, hed, unload_hed, fake_scribble, mlsd, unload_mlsd, midas, midas_normal, unload_midas, leres, unload_leres, openpose, openpose_hand, unload_openpose, uniformer, unload_uniformer, pidinet, unload_pidinet
import imutils
from PIL import Image
import os
import subprocess
import time

from base64 import b64encode

In [4]:
# %cd /content/stable-diffusion-webui/
# !COMMANDLINE_ARGS="--api	--enable-insecure-extension-access	--disable-safe-unpickle --no-half-vae --xformers --reinstall-xformers --enable-insecure-extension- --gradio-queue --cloudflared" REQS_FILE="requirements.txt" python launch.py


In [5]:
height, width = 512, 712
ratio = max(height, width) / 512

In [6]:
import cv2 as cv



class controlnetRequest():
    def __init__(self, b64hint, size, **kwargs ):
        self.url = "http://localhost:7860/controlnet/txt2img"
        height, widht = size

        
        
        
        # get the parameters from the kwargs or use default value
        self.body = {
            "prompt": kwargs.get("prompt", ""),
            "negative_prompt": kwargs.get("negative_prompt", ""),
            "seed": kwargs.get("seed", -1),
            "subseed": kwargs.get("subseed", -1),
            "subseed_strength": kwargs.get("subseed_strength", 0),
            "batch_size": kwargs.get("batch_size", 1),
            "n_iter": kwargs.get("n_iter", 1),
            "steps": kwargs.get("steps", 30),
            "cfg_scale": kwargs.get("cfg_scale", 7),
            "width": kwargs.get("width", 512),
            "height": kwargs.get("height", 512),
            "restore_faces": kwargs.get("restore_faces", True),
            "eta": kwargs.get("eta", 0),
            "sampler_index": kwargs.get("sampler_index", "Euler a"),
            "controlnet_input_image": [b64hint],
            "controlnet_module": kwargs.get("controlnet_module", "none"),
            "controlnet_model": kwargs.get("controlnet_model", "control_canny [e3fe7712]"),
            "controlnet_guidance": kwargs.get("controlnet_guidance", 1.0)
        }
            

    def sendRequest(self):
        r = requests.post(self.url, json=self.body)
        return r.json()




def create_hint(init_image, processor_model, controlnet_processor_res=512, thresholdA=None, thresholdB=None):
    """ Create a hint image from the initial image and the processor model
    Args:
        init_image: np array of the initial image 
        processor_model: the processor model, one of the following: 
            - canny
            - simple_scribble
            - hed
            - fake_scribble
            - mlsd
            - midas
            - midas_normal
            - leres
            - openpose
            - openpose_hand
            - uniformer
        thresholdA: the threshold A
        thresholdB: the threshold B
    """
    if processor_model == "canny":
        return canny(init_image, controlnet_processor_res, thresholdA, thresholdB)
    elif processor_model == "hed":
        return hed(init_image, controlnet_processor_res)
    elif processor_model == "mlsd":
        return mlsd(init_image, controlnet_processor_res, thresholdA, thresholdB)
    elif processor_model == "depth":
        return midas(init_image, controlnet_processor_res, np.pi * 2.0)
    elif processor_model == "normal_map":
        return midas_normal(init_image, controlnet_processor_res, np.pi * 2.0, thresholdA)
    elif processor_model == "depth_leres":
        return leres(init_image, controlnet_processor_res, np.pi * 2.0, thresholdA, thresholdB)
    elif processor_model == "openpose":
        return openpose(init_image, controlnet_processor_res, False)
    elif processor_model == "openpose_hand":
        return openpose_hand(init_image, controlnet_processor_res, True)
    elif processor_model == "fake_scribble":
        return fake_scribble(init_image, controlnet_processor_res)
    elif processor_model == "segmentation":
        return uniformer(init_image, controlnet_processor_res)
    elif processor_model == "pidinet":
        return pidinet(init_image, controlnet_processor_res)
    else:
        return None
    
    

def generate_controlnet(hint, **kwargs):
    """ Generate the controlnet meme by making a request to local api
    Args:
        hint: the hint image b64 encoded
        kwargs: the parameters of the controlnet
            - nb_steps: the number of steps
            - prompt: the prompt
            - guess_mode: false if we want to use the prompt, true if we want to use the guess
    Returns:
        List of images generated by controlnet
    """
    
    req = controlnetRequest(hint, (512,512), **kwargs)
    resp = req.sendRequest()
    print(resp)
    
    return resp['images']
    

In [7]:
%%file /content/make_video.sh
# https://gist.github.com/anguyen8/d0630b6aef6c1cd79b9a1341e88a573e




#!/bin/bash

# Anh Nguyen <anh.ng8@gmail.com>
# 2016-04-30
# MIT License

# This script takes in same-size images from a folder and make a crossfade video from the images using ffmpeg.
# Make sure you have ffmpeg installed before running.

# The output command looks something like the below, but for as many images as you have in the folder.
# See the answer by LordNeckbeard at:
# http://superuser.com/questions/833232/create-video-with-5-images-with-fadein-out-effect-in-ffmpeg/1071748#1071748
#
#
# ffmpeg \
# -loop 1 -t 1 -i 001.png \
# -loop 1 -t 1 -i 002.png \
# -loop 1 -t 1 -i 003.png \
# -loop 1 -t 1 -i 004.png \
# -loop 1 -t 1 -i 005.png \
# -filter_complex \
# "[1:v][0:v]blend=all_expr='A*(if(gte(T,0.5),1,T/0.5))+B*(1-(if(gte(T,0.5),1,T/0.5)))'[b1v]; \
# [2:v][1:v]blend=all_expr='A*(if(gte(T,0.5),1,T/0.5))+B*(1-(if(gte(T,0.5),1,T/0.5)))'[b2v]; \
# [3:v][2:v]blend=all_expr='A*(if(gte(T,0.5),1,T/0.5))+B*(1-(if(gte(T,0.5),1,T/0.5)))'[b3v]; \
# [4:v][3:v]blend=all_expr='A*(if(gte(T,0.5),1,T/0.5))+B*(1-(if(gte(T,0.5),1,T/0.5)))'[b4v]; \
# [0:v][b1v][1:v][b2v][2:v][b3v][3:v][b4v][4:v]concat=n=9:v=1:a=0,format=yuv420p[v]" -map "[v]" out.mp4

#----------------------------------------------------------------
# SETTINGS
input_dir="/content/video"  # Replace this by a path to your folder /path/to/your/folder
n_files=3                        # Replace this by a number of images
files=`ls ${input_dir}/*.jpeg | head -${n_files}`  # Change the file type to the correct type of your images
output_file="/content/video.mp4"           # Name of output video
crossfade=0.9                     # Crossfade duration between two images
#----------------------------------------------------------------

# Making an ffmpeg script...
input=""
filters=""
output="[0:v]"

i=0

for f in ${files}; do
  input+=" -loop 1 -t 1 -i $f"

  next=$((i+1))
  if [ "${i}" -ne "$((n_files-1))" ]; then
    filters+=" [${next}:v][${i}:v]blend=all_expr='A*(if(gte(T,${crossfade}),1,T/${crossfade}))+B*(1-(if(gte(T,${crossfade}),1,T/${crossfade})))'[b${next}v];"
  fi

  if [ "${i}" -gt "0" ]; then
    output+="[b${i}v][${i}:v]"
  fi

  i=$((i+1))
done

output+="concat=n=$((i * 2 - 1)):v=1:a=0,format=yuv420p[v]\" -map \"[v]\" ${output_file}"

script="ffmpeg ${input} -filter_complex \"${filters} ${output}"

echo ${script}

# Run it
eval "${script}"

Writing /content/make_video.sh


In [8]:
def generate_video_ffmep(img1_url, img2_url):
  !mkdir -p /content/video

  !wget -O /content/video/im1.jpeg {img1_url}
  !wget -O /content/video/im2.jpeg {img2_url}
  !cp /content/video/im1.jpeg /content/video/im3.jpeg
  !yes | bash /content/make_video.sh

  return '/content/video.mp4'



In [9]:
generate_video_ffmep("https://storage.googleapis.com/control-meme-public/meme_variation_20230304-065624.jpeg",
                     "https://storage.googleapis.com/control-meme-public/meme_variation_20230304-065305.jpeg")

--2023-03-05 19:35:24--  https://storage.googleapis.com/control-meme-public/meme_variation_20230304-065624.jpeg
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.24.128, 142.250.4.128, 142.251.10.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.24.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41790 (41K) [image/jpeg]
Saving to: ‘/content/video/im1.jpeg’

/content/video/im1. 100%[===================>]  40.81K  --.-KB/s    in 0.05s   

2023-03-05 19:35:24 (819 KB/s) - ‘/content/video/im1.jpeg’ saved [41790/41790]

--2023-03-05 19:35:24--  https://storage.googleapis.com/control-meme-public/meme_variation_20230304-065305.jpeg
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.24.128, 142.250.4.128, 142.251.10.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.24.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43448 (42K) [image/jpeg]
Savin

'/content/video.mp4'

# Finaly, run this cell and click on the outputed meme.koll.ai?gpuURL=XXX to be redirected to the website or copy https://XXX in the colabsessionlink textfield. 
#Wait for sd-webui API to be started before making generation, logs are in /content/output_webui.txt ( ~3 min)
(ready when 127.0.0.1 url and cloudfare url are displayed at the end of the file)

In [ ]:
%cd /content/stable-diffusion-webui

# run localtunnel
with open('/content/output_tunnel.txt', 'w') as f:
    proc = subprocess.Popen(['lt', '--port', '5000'], stdout=f)



os.environ['COMMANDLINE_ARGS'] = "--api --allow-code	--enable-insecure-extension-access	--disable-safe-unpickle --no-half-vae --xformers --reinstall-xformers --enable-insecure-extension- --gradio-queue --cloudflared" 
os.environ['REQS_FILE'] = "requirements.txt"
# del os.environ['REQS_FILE']

with open('/content/output_webui.txt', 'w') as f:
    proc_automatic = subprocess.Popen(['python', '-u', 'launch.py'], stdout=f, stderr=f)


time.sleep(3)

app = Flask(__name__)
CORS(app)

params = dict()

API_URL  = ""
BACK_URL = "https://controle-meme-back-n7bkntouja-uc.a.run.app"



def cv_to_b64(cv_image):
    _, im_arr = cv2.imencode('.jpg', cv_image)  # im_arr: image in Numpy one-dim array format.
    im_bytes = im_arr.tobytes()
    im_b64 = base64.b64encode(im_bytes)
    return im_b64

#Route to generate the hint according to processor, thresholdA and thresholdB
@app.route('/hint/', methods=['POST'])
def hint():
    params = request.get_json()
    meme_url = params['controlnet_basememe_url']
    processor_model = params['controlnet_module']
    thresholdA = params['controlnet_threshold_a']
    thresholdB = params['controlnet_threshold_b']
    
    # read the image from the url to cv2
    base_meme = imutils.url_to_image(meme_url)    
    hint = create_hint(base_meme, processor_model, thresholdA=thresholdA, thresholdB = thresholdB)
    hint = hint[0]
    # hint = Image.fromarray(hint)
    print(type(hint))
    print(hint)
    
    # # convert the hint to bytes
    # buffered = BytesIO()
    # hint.save(buffered, format="JPEG")
    
    # # encode the bytes to base64
    # img_str = base64.b64encode(buffered.getvalue())

    im_b64 = cv_to_b64(hint)
    
    return im_b64


@app.route('/', methods=['POST'])
def index():
    global params

    params = request.get_json()
    meme_url = params['controlnet_basememe_url']
    processor_module = params['controlnet_module']
    thresholdA = params['controlnet_threshold_a']
    thresholdB = params['controlnet_threshold_b']
        
    # read the image from the url to cv2
    base_meme = imutils.url_to_image(meme_url)    
    hint = create_hint(base_meme, processor_module, thresholdA=thresholdA, thresholdB = thresholdB)
    hint = hint [0]
    hint_b64 = cv_to_b64(hint)
    hint_b64 = hint_b64.decode("utf-8")

    # todo: send request to sdwebui api to get the generated image
    try: 
        last_image = generate_controlnet(hint_b64, **params)
    except requests.exceptions.ConnectionError:
        return "Automatic api subprocess is not fully started yet, please try again in a minute", 500
      
    last_image_b64 = last_image[0]
    
    # with open('./last_meme.jpeg', 'w') as f:
    #   Image.open(BytesIO(base64.b64decode(last_image))).save(f, format="JPEG")
    print('START')
    print(last_image_b64)
    print(type(last_image_b64))
    print('END')
    # with open("./last_meme.jpeg", "wb") as f:
    #    f.write(last_image_b64)


    decoded_data = base64.b64decode(last_image_b64)
    np_data = np.fromstring(decoded_data,np.uint8)
    img = cv2.imdecode(np_data,cv2.IMREAD_UNCHANGED)

    cv2.imwrite('./last_meme.jpeg', img)



    return last_image_b64



@app.route('/save_last/')
def save_last():
    global params

    # last_image = Image.open('./last_meme.jpeg')
    # buffered = BytesIO()
    # last_image.save(buffered, format="JPEG")
    # img_str = base64.b64encode(buffered.getvalue())

    last_image = cv2.imread('./last_meme.jpeg')
    img_str = cv_to_b64(last_image)

    data = dict(
        memeID = params['uuid'],
        imageb64=img_str.decode("utf-8"),
        prompt=params.get('prompt', 'no prompt'),
        nb_steps=params.get('nb_steps', 'no data'),
        parent_url = params.get('parent_url')
    )
    url = f'{BACK_URL}/api/save_variation/'
    print('url', url)
    requests.post(url, json = data)

    return "ok"


@app.route('/generate_video/', methods=['POST'])
def generate_video():
    params = request.get_json()
    
    parent_url = params['parent_url']
    variation_url = params['variation_url']

    path_video = generate_video_ffmep(parent_url, variation_url)

    return send_file(path_video)




@app.route('/hello/')
def hello():    
    return "hello"

# todo: add route to check if automatic api is started




if __name__ == '__main__':
  with open('/content/output_tunnel.txt', 'r') as f:
      contents = f.read()
      API_URL = contents.split(' ')[-1]
      protocol = API_URL.split(':')[0]
      url = API_URL.split('/')[-1]

      print(API_URL)

      print('\n'*2)
      print(f'Everything is ready! Click on the this link to be redirected to koll.ai. Don\'t close this tab!')
      print(f'>>>>  https://meme.koll.ai?gpuURL={url}  <<<<')

  app.run()

/content/stable-diffusion-webui
https://wet-books-knock-34-143-194-44.loca.lt




Everything is ready! Click on the this link to be redirected to koll.ai. Don't close this tab!
>>>>  https://meme.koll.ai?gpuURL=wet-books-knock-34-143-194-44.loca.lt
  <<<<
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [05/Mar/2023 19:48:58] "OPTIONS /hint/ HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Mar/2023 19:48:58] "POST /hint/ HTTP/1.1" 200 -


<class 'numpy.ndarray'>
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


INFO:werkzeug:127.0.0.1 - - [05/Mar/2023 19:50:24] "OPTIONS / HTTP/1.1" 200 -
<ipython-input-13-03b86ebd7d6c>:100: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  np_data = np.fromstring(decoded_data,np.uint8)
INFO:werkzeug:127.0.0.1 - - [05/Mar/2023 19:51:03] "POST / HTTP/1.1" 200 -


{'images': ['...'], 'parameters': {'enable_hr': False, 'denoising_strength': 0, 'firstphase_width': 0, 'firstphase_height': 0, 'hr_scale': 2.0, 'hr_upscaler': None, 'hr_second_pass_steps': 0, 'hr_resize_x': 0, 'hr_resize_y': 0, 'prompt': 'Cute cat digital painting, masterpiece', 'styles': None, 'seed': -1, 'subseed': -1, 'subseed_strength': 0.0, 'seed_resize_from_h': -1, 'seed_resize_from_w': -1, 'sampler_name': None, 'batch_size': 1, 'n_iter': 1, 'steps': 30, 'cfg_scale': 7.0, 'width': 512, 'height': 512, 'restore_faces': True, 'tiling': False, 'negative_prompt': '', 'eta': 0.0, 's_churn': 0.0, 's_tmax': None, 's_tmin': 0.0, 's_noise': 1.0, 'override_settings': None, 'override_settings_restore_afterwards': True, 'script_args': [], 'sampler_index': 'Euler a', 'script_name': None}, 'info': '{"prompt": "Cute cat digital painting, masterpiece", "all_prompts": ["Cute cat digital painting, masterpiece"], "negative_prompt": "", "all_negative_prompts": [""], "seed": 338820939, "all_seeds": [3

INFO:werkzeug:127.0.0.1 - - [05/Mar/2023 19:51:22] "OPTIONS /save_last/ HTTP/1.1" 200 -


url https://controle-meme-back-n7bkntouja-uc.a.run.app/api/save_variation/


INFO:werkzeug:127.0.0.1 - - [05/Mar/2023 19:51:30] "GET /save_last/ HTTP/1.1" 200 -


# Tests (ignore)

In [ ]:
meme_url = "https://storage.googleapis.com/control-meme-public/gigachad.jpg"

image = Image.open(requests.get(meme_url, stream=True).raw)
image

#get controlnet hint image
hint = create_hint(image, 'canny')
hint

params = {"prompt": "Gigachad Thanos",
          "hint": hint,
          "num_inference_steps": 30,
          "seed": -1
        }

b64 = generate_controlnet(**params)
b64

In [ ]:

meme_url = "https://storage.googleapis.com/control-meme-public/Salt-Bae-001.jpg"
processor_module = "canny"
thresholdA = 40
thresholdB = 120

# read the image from the url to cv2
base_meme = imutils.url_to_image(meme_url)    
hint = create_hint(base_meme, processor_module, thresholdA=thresholdA, thresholdB = thresholdB)
hint = Image.fromarray(hint)

# convert the hint to bytes
buffered = BytesIO()
hint.save(buffered, format="JPEG")

# encode the bytes to base64
hint_b64 = base64.b64encode(buffered.getvalue()).decode("utf-8")

# todo: send request to sdwebui api to get the generated image
last_image = generate_controlnet(hint_b64, **params)
last_image = last_image[0]

{'images': ['...'], 'parameters': {'enable_hr': False, 'denoising_strength': 0, 'firstphase_width': 0, 'firstphase_height': 0, 'hr_scale': 2.0, 'hr_upscaler': None, 'hr_second_pass_steps': 0, 'hr_resize_x': 0, 'hr_resize_y': 0, 'prompt': 'shrek', 'styles': None, 'seed': -1, 'subseed': -1, 'subseed_strength': 0.0, 'seed_resize_from_h': -1, 'seed_resize_from_w': -1, 'sampler_name': None, 'batch_size': 1, 'n_iter': 1, 'steps': 30, 'cfg_scale': 7.0, 'width': 512, 'height': 512, 'restore_faces': True, 'tiling': False, 'negative_prompt': '', 'eta': 0.0, 's_churn': 0.0, 's_tmax': None, 's_tmin': 0.0, 's_noise': 1.0, 'override_settings': None, 'override_settings_restore_afterwards': True, 'script_args': [], 'sampler_index': 'Euler a', 'script_name': None}, 'info': '{"prompt": "shrek", "all_prompts": ["shrek"], "negative_prompt": "", "all_negative_prompts": [""], "seed": 3311761444, "all_seeds": [3311761444], "subseed": 2444292267, "all_subseeds": [2444292267], "subseed_strength": 0.0, "width":